In [ ]:
!jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10 

In [ ]:
pip install -r requirements.txt

In [ ]:
import pymongo
client = pymongo.MongoClient("mongodb://USERNAME:PASSWORD@IP:27017/COLLECTION")  # defaults to port 27017
db=client["<DB_NAME>"]
collection=db["<COLLECTION_NAME>"] 


allData=[]
for i in collection.find({'do_not_process_flag': '0',
                          'is_similar': '0' }):
    allData.append(i)
print("Lenght of all data ",len(allData))

In [ ]:
import pandas as pd
columns=['_id', 'idx', 'index',
        #Article Metadata
        "id","link","article","title","datetime","media",
        
         
         
         # Named Entities from IBM
         'person_ibm','place_ibm','organization_ibm','quantity_ibm','got_entities_from_ibm',
         'person_ibm_title','place_ibm_title','organization_ibm_title','quantity_ibm_title','got_entities_from_ibm_title',
         
         #Similarity Checker
         'idx_with_similar_title',     
         
         
         "do_not_process_flag", "similar_article", "is_similar","processed_for_similarity_check",
         "error_IBM_in_title", "error_IBM_in_article",
         
         # CIF Fields
         "num_of_victims","victim_list","len_victim_list","flagged_victim_list",
         "pv1_name","pv1_gender","pv1_age","pv1_nationality","pv1_address","pv1_education","pv1_occupation","pv1_guardian",
         "pv2_name","pv2_gender","pv2_age","pv2_nationality","pv2_address","pv2_education","pv2_occupation","pv2_guardian",
         "pv3_name","pv3_gender","pv3_age","pv3_nationality","pv3_address","pv3_education","pv3_occupation","pv3_guardian",
         "pv4_name","pv4_gender","pv4_age","pv4_nationality","pv4_address","pv4_education","pv4_occupation","pv4_guardian",
         "pv5_name","pv5_gender","pv5_age","pv5_nationality","pv5_address","pv5_education","pv5_occupation","pv5_guardian",
         "pv6_name","pv6_gender","pv6_age","pv6_nationality","pv6_address","pv6_education","pv6_occupation","pv6_guardian",
         "pb1_name","pb1_gender","pb1_age","pb1_nationality","pb1_address","pb1_education","pb1_occupation",
         "pb2_name","pb2_gender","pb2_age","pb2_nationality","pb2_address","pb2_education","pb2_occupation",
         "pb3_name","pb3_gender","pb3_age","pb3_nationality","pb3_address","pb3_education","pb3_occupation",
         "pb4_name","pb4_gender","pb4_age","pb4_nationality","pb4_address","pb4_education","pb4_occupation",
         "pb5_name","pb5_gender","pb5_age","pb5_nationality","pb5_address","pb5_education","pb5_occupation",
         "pb6_name","pb6_gender","pb6_age","pb6_nationality","pb6_address","pb6_education","pb6_occupation",
         
         'cif_extracted_flag',

         "promise_to_victims","purpose_to_travel","time_taken",
         "traffickers","form_of_exploitation","len_token","long_articles_FLAG","updated_in_database"]
result_df=pd.DataFrame(columns=columns)

# Check if file is there or not, if file dont exist create one
try:
    f=open("result_df.csv",'r')
    f.close()
except:
    print("File doesnot exits,\n Creating file ")
    result_df.to_csv("result_df.csv")
    

In [ ]:
result_df['updated_in_database']=0

In [ ]:
#IBM WATSON API CODE

import json
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1  import Features, EntitiesOptions

authenticator = IAMAuthenticator('{apikey}')
natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2020-08-01',
    authenticator=authenticator
)

natural_language_understanding.set_service_url("url")


def get_entities(article):
    """
    This function returns the entities from the article by sending article to the IBM API.
    The returns attributes are person, location, quantity, organization. 
    """
    response = natural_language_understanding.analyze(
        text=article,
        features=Features(entities=EntitiesOptions(sentiment=True,limit=None))).get_result()

    #print(json.dumps(response, indent=2))

    #Entitiy list
    person=[]
    location=[]
    quantity=[]
    organization=[]

    for i in response['entities']:
      
      if i['type']=='Person':
        person.append(i['text'].lower().strip())
      if i['type']=='Location':
        location.append(i['text'].lower().strip())
      if i['type']=='Quantity' :
        quantity.append(i['text'].lower().strip())
      if i['type']=='Organization' :
        organization.append(i['text'].lower().strip())
    
    return person, location, quantity, organization

In [ ]:

import torch
from transformers import BertForQuestionAnswering
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [ ]:

from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [ ]:
import re
def removeSpaces(my_str):
  _RE_COMBINE_WHITESPACE = re.compile(r"\s+")
  return  _RE_COMBINE_WHITESPACE.sub(" ", my_str).strip()


In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

In [ ]:
model.to(device)

In [ ]:
def answer_question(question, answer_text):
    """
    This function will take the article (answer_text) and the question. The function will ask
    model the question by passing both question and the article. The model returns the score and the answer which is
    also the returnig attributes of the function. 
    """
    input_ids = tokenizer.encode(question, answer_text)
    sep_index = input_ids.index(tokenizer.sep_token_id)
    num_seg_a = sep_index + 1
    num_seg_b = len(input_ids) - num_seg_a
    segment_ids = [0]*num_seg_a + [1]*num_seg_b
    assert len(segment_ids) == len(input_ids)
    
    
    inputIDS = torch.tensor([input_ids], dtype=torch.long).to(device,dtype=torch.long)
    segmentIDS=torch.tensor([segment_ids], dtype=torch.long).to(device,dtype=torch.long)

    outputs = model(inputIDS, # The tokens representing our input text.
                                    token_type_ids=segmentIDS ) # The segment IDs to differentiate question from answer_text
    start_scores=outputs[0].detach().cpu()             
    end_scores=outputs[1].detach().cpu()
    START_SCORES=max(torch.sigmoid(start_scores).detach().numpy().tolist()[0])
    
    # We only consider the answer if and only if the score is greater than 90%
    # This makes sure we dont have to process any unncessary answer. 
    if  START_SCORES> 0.90:
      answer_start = torch.argmax(start_scores)
      answer_end = torch.argmax(end_scores)
      tokens = tokenizer.convert_ids_to_tokens(input_ids)
      answer = tokens[answer_start]
      for i in range(answer_start + 1, answer_end + 1):
          
          
          if tokens[i][0:2] == '##':
              answer += tokens[i][2:]
          else:
              answer += ' ' + tokens[i]
      answer=removeSpaces(answer)
      return answer, START_SCORES
    else:
      return "NA",0.00

In [ ]:

from collections import Counter 
import time

def most_frequent(List): 
    """
    We have two mechanism to select the answer, one is to select the answer with the maximum score,
    and the other one is to select the most frequent answer.
    Note: We are currently using answer with maximum score. 
    """
    if len(List)>0:
      occurence_count = Counter(List) 
      return occurence_count.most_common(1)[0][0],occurence_count.most_common(1)[0][1] 
      # returns Most common item and the frequency

In [ ]:
import re
# to wrap the printed layout
import textwrap
wrapper = textwrap.TextWrapper(width=90) 

########### HANDLE LONGER LENGTH ARTICLES ####################
def get_token_len(question, answer_text):
    #print(" ** -- Article is longer in length -- **")
    """
    This function calculate the token length 
    and the number of parts in which article needs to be divided
    """
    token_len=len(tokenizer.encode(question, answer_text))
    number_of_short_text=int( (token_len/512) +1)
    return token_len,number_of_short_text # return the token length of the TEXT


def find_prev_period(art,pos): 
    """
    we need to send the complete sentence hence required to break the longer text,
    with smaller sentences having period(.) in its ends
    """
    for p in range(pos,0,-1):
        try:
            if art[p]=='.':
                return p #return the previous position of period
        except:#in the case of no period, return the original pos
              return pos

def break_longer_artilces(long_text,number_of_short_text):
  """
  This article breaks the longer article into smaller parts.
  The number of smaller part = number_of_short_text
  """
  print("-- ** article is cutting in half ** -- ")
  short_article_parts=[]
  cutoff=len(long_text)/number_of_short_text
  for i in range(number_of_short_text):
    if i==0:
      start= int(i*cutoff)
      end=find_prev_period (long_text, int( (i+1)*cutoff) )
    else:
      start=find_prev_period(long_text, int(i*cutoff) )
      end= find_prev_period (long_text, int( (i+1)*cutoff) )
    
    short_article_parts.append(long_text[start:end])

  return short_article_parts # return the dictionary with shorter articlees
 
def remove_pv(article, pv):
    """
    To find multiple victims, we need to remove founded victim names, this function removes
    the name of victim and return the articles.
    """
    try:
        for i in re.finditer(pv,article):
        
            s=int(i.start())
            e=int(i.end())
            first=article[:s]
            second=article[e:]
            new_article=first+second
            if new_article:
              return new_article
            if not new_article:
              return article
    except:
            return article
from collections import OrderedDict 
def re_modify_ibm_ner_list(x):

  #""" there could be Keshav and Keshav Bahadur,   
  #this function will remove the first name if the complete name is presented """
  rep_list=[]
  for i in range(len(x)):
    for j in range(len(x)):
      if i==j:
        #print(x[i],x[j])
        continue
      else:
        if x[j] in str(x[i].split()[0]):
          print(x[j] ,"->", str(x[i]))
          rep_list.append(x[j])
  new_list=list(set(x).difference(set(rep_list)))
  return  list(OrderedDict.fromkeys(new_list))

import re 
  
def clean_title_and_url(text): 
    try: # some articles might not have url at all
      regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
      url = re.findall(regex,text)  
      u=[x[0] for x in url] 
      start=text.find(u[0])+len(u[0])
      cleaned_txt=text[start:]
      return cleaned_txt
    except:
      return text




In [ ]:
from collections import OrderedDict 


def find_multiple_victims(article, victimSuspectFlag):
  #vic_sus_AnswerList = victim or suspect Answer list
  global vic_sus_AnswerList, count
  global ibm_person_list,location, quantity, organization
  
  print(f"###########  Loop is : {count} #################\n")
  print(f"")
  print(f"{count} Answer list from model is: {vic_sus_AnswerList}\n")
  count+=1
  if count>10 or  (count>len(ibm_person_list)): # if count is > 10, it means we are in the loop so it is better to end the program.
    return "" 
  
  print(f"\n the value of victimSuspectFlag is {victimSuspectFlag}")
  
  if victimSuspectFlag==0: # For identifying victims
    print("---- searching for victims ---")
    questions=["What is the name of victim? ",
                  "Who was kidnapped?",
                  "Who was the victim?",
                  "Who was the survivor?",
                  "Who were abducted?",
                  "Who were rescued?",
                  "Who was trafficked?",
                  "Who has gone missing?",
                  "Who was sold?",
                  "Who was cheated?"]
  if victimSuspectFlag==1: # For identifying traffickers
    print("---SEARCHING FOR TRAFFICKERS/SUSPECTS----")
    questions=[f"Who SOLD persons?",
                f"Who sold the missing persons?",
                f"Who sold the rescued persons?",
                f"Who was arrested?",
                f"Who was convicted?",
                  "Who was the trafficker?",
                  "Who was the kidnapper?",
                  "Who was arrested?",
                  "Who sold the victim?",
                  "Who was the abductor?",  
                  "Who was the suspect?"]  
  #identifying victims from BERT  model
  ans_list=[]
  score_list=[]
  for q in questions:
    try:
      """print("Right here--------->",q,article)
      print("<<-----------------")"""
      ans,score=answer_question(q,article)
      if ("[CLS]" not in ans) and ("[SEP]" not in ans) and ("NA" != ans):
        ans_list.append(ans)
        score_list.append(score)
    except:
      print(f"error at inside finding ans, {q}")
    
  ans_count=""
  ans_score=""
  pv="NA"
  if len(ans_list)>0:
    print(f"Answer List from model is : {ans_list}")
    # answer based on frequency/count
    common,freq=most_frequent(ans_list)
    if freq >1:
      ans_count=common
    #answer based on maximum
    ans_score=ans_list[score_list.index(max(score_list))]
    print(f"Answer based on Score is :{ans_score}")
    pv=ans_score
  else:
    ans_count=""
    ans_score=""
    pv="NA"
  
  print(f"\n Potential answer identified from model is : {pv}")

  ############################################# 

  #if no victims identified by model we get out of the function
  if pv=="NA":
    print("No more answers ")     
    print(vic_sus_AnswerList) 
    return vic_sus_AnswerList
   
  if pv!="NA":      
      print("\n Potential Answer, from the model is: ",pv)

      # CASE - I
      #if the length of the victim name is only one(first name), check in IBM names and get the full name from the Person Entity
      #it also solves the problem of getting half names, 
      #example : if model gives the 'man' as victim, from IBM Person entity list, it extract 'Manju
      if len(pv.split()) ==1:
        print("\n Only first name is given by model, checking for complete name in IBM name list")
        for person_names in ibm_person_list:
          if pv in person_names:
            print(f" Found complete name for {pv} -->: {person_names}")
            pv=person_names #repalce the pv from model to entity from IBM, such that it can be replaced
            break


      # CASE - II
      # if BERT model gives us the right victim name, then the victim name should be in the named entiies list in the IBM person list
      # (assumption- name must be in the IBM named entities list)->and this actually works !!!

      # we might want to check if PV is in IBM name list or not?
      # if yes then we might want to skip getting name entities from model answer.
      skip_ner_in_model_answer_flag=0
      for person_names in ibm_person_list:
                if pv in person_names: # checking if the pv is in the ibm person list or not
                  skip_ner_in_model_answer_flag=1
                  print("Potential answer from model is in IBM name list and hence Add to answer list and call recurive function")
                  vic_sus_AnswerList.append(pv)
                  print(f"Updated answer list is: {vic_sus_AnswerList} \n")
                  article=remove_pv(article,pv)
                  find_multiple_victims(article, victimSuspectFlag)
                  

      # CASE - III
      # #use IBM NER to recognize multiple victims in a single answer sentence given by the model
      # the output of the model might look like: "keshav bista , nabin bista ( 13 ) and bhuwan bhatta ( 14 )"
      # if we could extract, the named entities from above example, that means we got the name of the victim. 
      # Hence send the answer to IBM API for person name extraction. 
      # Note: some times IBM might give the name of Place also as a person entity.
      
      if skip_ner_in_model_answer_flag == 0:
               
                person=[] #save the person list from IBM api
                try:
                  # we need to add some text, in order for ibm api to work and identify the name entity(victims)
                  #newPV="The model gave following as the answer "+pv
                  newPV=f"The answer given by the machine is {pv}"
                  print(f"New text to send to IBM API:::--> {newPV}")
                  person,_,_,_=get_entities(newPV)
                  print(f"from ibm {person}") 
                except:
                  print("error in first try ")   

                # if the IBM api gives the name list then those should be added to the victim list      
                if len(person)>0: 
                  #we add it to our list                  
                  vic_sus_AnswerList=vic_sus_AnswerList+person
                  print(f"Answer list-- {person}--is extracted from IBM API \n\t and the new answer list is: ",vic_sus_AnswerList)


                  #now we need to remove those names from article and look again for other victims
                  for name in person:
                    article=remove_pv(article,name)
                  print("after removal of name, \n:",article)
                  
                  #call recursive function with removed victims artilces
                  find_multiple_victims(article, victimSuspectFlag)

    
      # for example like this : "keshav bista , nabin bista ( 13 ) and bhuwan bhatta ( 14 )"
      # IBM might not always work, hence we extract names based on separator such as comma ',' and 'AND'
      # this is based on the observation I have made so far looking into the answer as provided by the modell for multiple answers
      if ("," in pv) and (len(pv.split(",")) >1):
          print("Removing ',' :", pv.split(",")[0])
          
          try:
                      article=remove_pv(article,pv.split(",")[0])
          except:
            print(" error in removing , part")

          vic_sus_AnswerList.append(pv.split(",")[0])

          find_multiple_victims(article, victimSuspectFlag)

      elif ("and " in pv) and (len(pv.split("and")) >1):
          print("Removing 'and' :", pv.split("and")[0])
          #need try and exception here
          try:
                      article=remove_pv(article,pv.split("and")[0])
          except:
            print(" error in removing and part")

          try:
                      article=remove_pv(article,pv.split("and")[1])
          except:
            print(" error in removing and part")

          vic_sus_AnswerList.append(pv.split("and")[0])
          vic_sus_AnswerList.append(pv.split("and")[1])

          find_multiple_victims(article, victimSuspectFlag)

      else:
              
            print("\t\t ***--- IBM (longer text for multiple answer ) has not been used \n Splitting method has not been used. \n ")
            print("\t\t ***--- Calling recursion function from last part----- ")
            #print(" PV at this point is ,",pv)
            #print(" Article at last part: ",article)
            try:
              article=remove_pv(article,pv)
            except:
              return ""
            vic_sus_AnswerList.append(pv)
            find_multiple_victims(article, victimSuspectFlag)
      
      

In [ ]:
def get_victims_list(article, victimSuspectFlag):
  
  global vic_sus_AnswerList,flagged_victim_list
  global ibm_person_list,location, quantity, organization
  vic_sus_all=[] #after removing the non IBM person entities
  

  
  vl=find_multiple_victims(article.lower(), victimSuspectFlag)
  #print(f"vl ={vl}")
  for i in vic_sus_AnswerList:    
    for j in ibm_person_list:
      if i.strip()==j.strip():
          vic_sus_all.append(i) 

  if victimSuspectFlag==0:
    print(f"Victim AnswerList={vic_sus_AnswerList} ") 
  if victimSuspectFlag==1:
    print(f"Suspect AnswerList={vic_sus_AnswerList} ") 

  flagged_victim_list = list(set(vic_sus_AnswerList).difference(set(vic_sus_all)) )
  #flagged_victim_list = list(set(victim_list).difference(set(victim_list_all)) )
  #print("flagged victim before returning: ",list(OrderedDict.fromkeys( flagged_victim_list )))
  #return victim_list_all, flagged_victim_list
  return list(OrderedDict.fromkeys( vic_sus_all )), list(OrderedDict.fromkeys( flagged_victim_list ))

In [ ]:
def convertListToLower(lst):
  #we want to convert entities from IBM to lower case in order for them to be matched.
  lowerList=[]
  for i in lst:
    lowerList.append(i.lower().strip())
  return lowerList

In [ ]:
"""
1.0
"""


def get_numberOfVictims(finalVictimList,articleStory):

    
    
  #create a dynamic questions 
  questions=[f"How many victims were there?",
            f"How many people were rescued?",
            f"How many people went missing?"]

  ans_count=""
  ans_score=""
  ans_list=[]
  score_list=[]
  ans=''
  score=0.0


  for q in questions:
    try:
      ans,score=answer_question(q,articleStory)
      if ("[CLS]" not in ans) and ("[SEP]" not in ans) and ("NA" != ans):

        ans_list.append(ans)
        score_list.append(score)
        error=0
        error_remark=""
    except RuntimeError as err:
      print(f"error at address, {q}")
      error=1
      error_remark=err
      break     
  if len(ans_list)>0:
    # answer based on frequency/count
    common,freq=most_frequent(ans_list)
    if freq >1:
      ans_count=common
    #answer based on maximum
    ans_score=ans_list[score_list.index(max(score_list))]
  else:

      ans_count=''
  numberOfVictims=ans_score
  return numberOfVictims

"""

1.2 GENDER OF VICTIMS


"""


# Gender of Victims



import json
from urllib.request import urlopen
myKey='<key>' 

def genderAPI(name):
  url = "https://gender-api.com/get?key=" + myKey + f"&name={name}"
  response = urlopen(url)
  decoded = response.read().decode('utf-8')
  data = json.loads(decoded)
  return data["gender"] #Gender: male


def get_genderOfVictims(finalVictimList, articleStory):
    genderOfVictims={}
    for pv,i in zip(finalVictimList,range( len(finalVictimList) )):
        #create a dynamic questions 
        
        ans_count=""
        ans_score="", 0.0
        ans_list=[]
        score_list=[]
        ans=''
        score=0.0
        #print(pv)
        # Dynamic Question is created with the name of victim (not the one that the model predicted
        questions=[
                  f"What was the sexuality of {pv}?",
                  f"What was the gender of {pv} ?",
                  f"Is {pv} a boy or a girl?",
                  f"Is {pv} a male or a female?"]
        
        

        
        for q in questions:
          try:
            ans,score=answer_question(q,articleStory)
            if ("[CLS]" not in ans) and ("[SEP]" not in ans) and ("NA" != ans):
              if ('girl' in ans) or ('girls' in ans) or ('women' in ans) or ('female' in ans) or ('daughter' in ans):          
                ans='female'
              elif ('boy' in ans) or ('boys' in ans) or ('men' in ans) or ('man' in ans) or ('male' in ans) or ('son' in ans) :
                ans='male'
              else:
                
                continue
                
              ans_list.append(ans)
              score_list.append(score)
              error=0
              error_remark=""
          except RuntimeError as err:
            print(f"error at , {q}")
            error=1
            error_remark=err
            break
        
        
        if len(ans_list)>0:
          # answer based on frequency/count
          common,freq=most_frequent(ans_list)
          if freq >1:
            ans_count=common
          #answer based on maximum
          ans_score=ans_list[score_list.index(max(score_list))], max(score_list)
        else:
          try:
            ans=genderAPI(pv.split()[0])
            #df3['gender_api_use'][i]=1
            print("\t Use of gender api- ", pv," ->", ans)
            ans_score=ans, 0.99999999999
            ans_count=ans
          except:
            print('Error in GenderAPI')
      
        #print("Answer based on score->",ans_score)
        #print("Answere based on count->",ans_count)
        genderOfVictims[pv]=ans_score
    return genderOfVictims
"""
1.3 AGE

"""

def get_ageOfVictims(finalVictimList, articleStory):
    ageOfVictims={}
    for pv,i in zip(finalVictimList,range( len(finalVictimList) )):
      #create a dynamic questions 
      questions=[f"How old is  {pv}?",\
                f"What is the age of {pv}?"]

      ans_count=""
      ans_score="",0.0
      ans_list=[]
      score_list=[]
      ans='' 
      score=0.0
      
      ageOfVictims[pv]=0
      for q in questions:
        try:
          ans,score=answer_question(q,articleStory)
          if ("[CLS]" not in ans) and ("[SEP]" not in ans) and ("NA" != ans):
            
            ans_list.append(ans)
            score_list.append(score)
            error=0
            error_remark=""
        except RuntimeError as err:
          print(f"error at , {q}")
          error=1
          error_remark=err
          break     
      if len(ans_list)>0:
        # answer based on frequency/count
        common,freq=most_frequent(ans_list)
        if freq >1:
          ans_count=common
        #answer based on maximum
        ans_score=ans_list[score_list.index(max(score_list))], max(score_list)
      else:
                    
          ans_count=''
      ageOfVictims[pv]=ans_score
    print(f"age   ->",ageOfVictims)
    return ageOfVictims
"""

1. 4 NATIONALITY

"""
def get_nationalityOfVictims(finalVictimList, articleStory):
  nationalityOfVictims={}
  for pv,i in zip(finalVictimList,range( len(finalVictimList) )):
    #create a dynamic questions 
    questions=[f"What is the nationality of {pv}?",
              f"Which country {pv} belongs to?"]

    ans_count=""
    ans_score="",0.0
    ans_list=[]
    score_list=[]
    ans=''
    score=0.0
    
    
    for q in questions:
      try:
        ans,score=answer_question(q,articleStory)
        if ("[CLS]" not in ans) and ("[SEP]" not in ans) and ("NA" != ans):
          
          ans_list.append(ans)
          score_list.append(score)
          error=0
          error_remark=""
      except RuntimeError as err:
        print(f"error at nationality, , {q}")
        error=1
        error_remark=err
        break     
    if len(ans_list)>0:
      # answer based on frequency/count
      common,freq=most_frequent(ans_list)
      if freq >1:
        ans_count=common
      #answer based on maximum
      ans_score=ans_list[score_list.index(max(score_list))], max(score_list)
    else:
                  
        ans_count=''
    nationalityOfVictims[pv]=ans_score
  return nationalityOfVictims

"""

1.5 ADDRESS

"""

def get_addressOfVictims(finalVictimList,articleStory):

    addressOfVictims={}
    for pv,i in zip(finalVictimList,range( len(finalVictimList) )):
      #create a dynamic questions 
      questions=[f"Where do  {pv} lives??",
                f"What is the address of {pv}?",
                f"Where is {pv} from?"]

      ans_count=""
      ans_score="",0.0
      ans_list=[]
      score_list=[]
      ans=''
      score=0.0
      
      
      for q in questions:
        try:
          ans,score=answer_question(q,articleStory)
          if ("[CLS]" not in ans) and ("[SEP]" not in ans) and ("NA" != ans):
            
            ans_list.append(ans)
            score_list.append(score)
            error=0
            error_remark=""
        except RuntimeError as err:
          print(f"error at address, , {q}")
          error=1
          error_remark=err
          break     
      if len(ans_list)>0:
        # answer based on frequency/count
        common,freq=most_frequent(ans_list)
        if freq >1:
          ans_count=common
        #answer based on maximum
        ans_score=ans_list[score_list.index(max(score_list))], max(score_list)
      else:
                    
          ans_count=''
      addressOfVictims[pv]=ans_score
    return addressOfVictims

"""

1.6 OCCUPATION

"""
def get_occupationOfVictims(finalVictimList,articleStory):
    occupationOfVictims={}
    for pv,i in zip(finalVictimList,range( len(finalVictimList) )):
      #create a dynamic questions 
      questions=[f"What was the occupation of {pv}?",
                f"What was {pv} source of income?",
                f"What was job of {pv} ?"]

      ans_count=""
      ans_score="",0.0
      ans_list=[]
      score_list=[]
      ans=''
      score=0.0
     
      
      for q in questions:
        try:
          ans,score=answer_question(q,articleStory)
          if ("[CLS]" not in ans) and ("[SEP]" not in ans) and ("NA" != ans):
            
            ans_list.append(ans)
            score_list.append(score)
            error=0
            error_remark=""
        except RuntimeError as err:
          print(f"error at occupation   {q}")
          error=1
          error_remark=err
          break     
      if len(ans_list)>0:   
        #answer based on maximum
        ans_score=ans_list[score_list.index(max(score_list))] , max(score_list)
      else:
                    
          ans_count=''
      occupationOfVictims[pv]=ans_score
    #print(occupationOfVictims)
    return occupationOfVictims


"""

1.7 EDUCATION

"""
def get_educationOfVictims(finalVictimList,articleStory):
    educationOfVictims={}
    for pv,i in zip(finalVictimList,range( len(finalVictimList) )):
      #create a dynamic questions 
      questions=[f"What was the education of {pv}?",
                f"What was {pv} schooling?",             
                f"How far {pv} studied?"]

      ans_count=""
      ans_score="",0.0
      ans_list=[]
      score_list=[]
      ans=''
      score=0.0
     
      
      for q in questions:
        try:
          ans,score=answer_question(q,articleStory)
          if ("[CLS]" not in ans) and ("[SEP]" not in ans) and ("NA" != ans):
            
            ans_list.append(ans)
            score_list.append(score)
            error=0
            error_remark=""
        except RuntimeError as err:
          print(f"error at ,education {q}")
          error=1
          error_remark=err
          break     
    
      if len(ans_list)>0:   
        #answer based on maximum
        print(ans_list)
        print(score_list)
        ans_score=ans_list[score_list.index(max(score_list))], max(score_list)
        print("--------------------")
      else:
                    
          ans_count=''
      educationOfVictims[pv]=ans_score
    #print(educationOfVictims)
    return educationOfVictims

# Guardians / Parents

"""

1.9 Guardians / Parents

"""
def get_guardiansOfVictims(finalVictimList,articleStory):
  guardiansOfVictims={}
  for pv,i in zip(finalVictimList,range( len(finalVictimList) )):
    #create a dynamic questions 
    questions=[f"Who were the relatives of {pv}?",
              f"Who were the families pf {pv} ?",             
              ]

    ans_count=""
    ans_score="",0.0
    ans_list=[]
    score_list=[]
    ans=''
    score=0.0
    
    
    for q in questions:
      try:
        ans,score=answer_question(q,articleStory)
        if ("[CLS]" not in ans) and ("[SEP]" not in ans) and ("NA" != ans):
          
          ans_list.append(ans)
          score_list.append(score)
          error=0
          error_remark=""
      except RuntimeError as err:
        print(f"error at Guardians   {q}")
        error=1
        error_remark=err
        break     
  
    if len(ans_list)>0:   
      #answer based on maximum
      print(ans_list)
      print(score_list)
      ans_score=ans_list[score_list.index(max(score_list))], max(score_list)
      print("--------------------")
    else:
                  
        ans_count=''
    guardiansOfVictims[pv]=ans_score
  return guardiansOfVictims


"""

2.1 TRAFFICKERS

"""
def get_traffickers(finalVictimList,articleStory):
      traffickers=""
      
      #create a dynamic questions 
      questions=[f"Who SOLD persons?",
                f"Who sold the missing persons?",
                f"Who sold the rescued persons?",
                f"Who was arrested?",
                f"Who was convicted?",
                  "Who was the trafficker?",
                  "Who was the kidnapper?",
                  "Who was arrested?",
                  "Who sold the victim?",
                  "Who was the abductor?",
                  "Who was the suspect?"]

      ans_count=""
      ans_score=""
      ans_list=[]
      score_list=[]
      ans=''
      score=0.0
      
      
      for q in questions:
        try:
          ans,score=answer_question(q,articleStory)
          if ("[CLS]" not in ans) and ("[SEP]" not in ans) and ("NA" != ans):
            
            ans_list.append(ans)
            score_list.append(score)
            error=0
            error_remark=""
        except RuntimeError as err:
          print(f"error at TRAFFICKERS , {q}")
          error=1
          error_remark=err
          break     
      if len(ans_list)>0:
        # answer based on frequency/count
        common,freq=most_frequent(ans_list)
        if freq >1:
          ans_count=common
        #answer based on maximum
        ans_score=ans_list[score_list.index(max(score_list))]
      else:
                    
          ans_count=''
      traffickers=ans_score
      return traffickers

"""

2.2 PROMISE/PERSUADE

"""
def get_promiseToVictims(finalVictimList,articleStory):
  promiseToVictims=[]

  
  questions=["How did trafficker persuade the victim?",
            f"What was the promise made to the victim?",
            f"What was the promise made to the survivor?" ]

  ans_count=""
  ans_score=""
  ans_list=[]
  score_list=[]
  ans=''
  score=0.0


  for q in questions:
    try:
      ans,score=answer_question(q,articleStory)
      if ("[CLS]" not in ans) and ("[SEP]" not in ans) and ("NA" != ans):
        
        ans_list.append(ans)
        score_list.append(score)
        error=0
        error_remark=""
    except:
      print(f"error at: {q}")
      error=1
      error_remark="Error happend while getting answer"
      break     
  if len(ans_list)>0:
    # answer based on frequency/count
    common,freq=most_frequent(ans_list)
    if freq >1:
      ans_count=common
    #answer based on maximum
    ans_score=ans_list[score_list.index(max(score_list))]
  else:                  
      ans_count=''
  promiseToVictims.append(ans_score)
  #print(promiseToVictims)   
  #nationalityOfVictims[pv]=ans_score
  return promiseToVictims
        
"""

3.1 PURPOSE/ REASON TO LEFT HOME

"""
def get_purposeToTravel(finalVictimList,articleStory):
     
    purposeToTravel=[]
    for pv,i in zip(finalVictimList,range( len(finalVictimList) )):
      #create a dynamic questions 
      questions=[f" Why did {pv} left home?",
                f"What was the main purpose for which {pv} leave the home? ",
                f"What was the main reason for which {pv} travel?"]

      ans_count=""
      ans_score=""  
      ans_list=[]
      score_list=[]
      ans=''
      score=0.0
      #print(pv)
      
      for q in questions:
        try:
          ans,score=answer_question(q,articleStory)
          if ("[CLS]" not in ans) and ("[SEP]" not in ans) and ("NA" != ans):
            
            ans_list.append(ans)
            score_list.append(score)
            error=0
            error_remark=""
        except RuntimeError as err:
          print(f"error at {pv}, {q}")
          error=1
          error_remark=err
          break     
      if len(ans_list)>0:
        # answer based on frequency/count
        common,freq=most_frequent(ans_list)
        if freq >1:
          ans_count=common
        #answer based on maximum
        ans_score=ans_list[score_list.index(max(score_list))]
      else:
                    
          ans_count=''
      purposeToTravel.append(ans_score)
    #print(purposeToTravel) 
    return purposeToTravel
"""
4.5 Exploitation
"""

def get_formOfExploitation(finalVictimList,articleStory):
    
    formOfExploitation=[]
    for pv,i in zip(finalVictimList,range( len(finalVictimList) )):
      #create a dynamic questions 
      questions=[f"How was {pv} exploited?",
                 f"How was {pv} abused? ",
                 f"What was {pv} forced to do?"]

      ans_count=""
      ans_score=""  
      ans_list=[]
      score_list=[]
      ans=''
      score=0.0
      #print(pv)
      
      for q in questions:
        try:
          ans,score=answer_question(q,articleStory)
          if ("[CLS]" not in ans) and ("[SEP]" not in ans) and ("NA" != ans):
            
            ans_list.append(ans)
            score_list.append(score)
            error=0
            error_remark=""
        except RuntimeError as err:
          print(f"error at 'Form of expolitation', {q}")
          error=1
          error_remark=err
          break     
      if len(ans_list)>0:
        # answer based on frequency/count
        common,freq=most_frequent(ans_list)
        if freq >1:
          ans_count=common
        #answer based on maximum
        ans_score=ans_list[score_list.index(max(score_list))]
      else:
                    
          ans_count=''
      formOfExploitation.append(ans_score)
    #print(purposeToTravel) 
    return formOfExploitation

In [ ]:
#function to insert into database, return whether the status is true or false.
def insertInMONGO(result):
    global collection
    print(collection)
    new_dict=result.copy()
    try:
        del new_dict['_id']
    except KeyError:
        pass
    #print( result["_id"])
    status=collection.update_one({'idx' : result['idx']}, 
                                {'$set' : new_dict},
                                 upsert= False )
    return status.acknowledged

In [ ]:
def remove_URL(sample):
    """Remove URLs from a sample string"""
    return re.sub(r"http\S+", "", sample)
import ast
def convert_to_List(txt):
    return ast.literal_eval(txt)

In [ ]:
# ##########################################################################################
import datetime
from itertools import chain 
long_articles=0

for  i in range(len(allData)) :
        start_time = datetime.datetime.now()
              
        
        result_dict=allData[i]
        
        #1.1 Victims Name
        #check for article token length
        articleStory= remove_URL(result_dict['article'].lower())
        token_length, number_of_short_text = get_token_len ( " What is the name of the victim?", articleStory)
        print("\n\n---------------------------------------------------------------------------------------#####")
        print(f" {i}, Token length is {token_length}--------------------------------------------------------")
        
        # Get the named entities for article,as saved in MongoDB during crawling.
        
        ibm_person_list= convert_to_List(result_dict["person_ibm"])
        location=convert_to_List(result_dict["place_ibm"])
        quantity=convert_to_List(result_dict["quantity_ibm"])
        organization=convert_to_List(result_dict["organization_ibm"])
        
        # convert the named entities to the lower such that, comparing results becomes easy
        
        ibm_person_list= convertListToLower(ibm_person_list )
        location=convertListToLower(location )
        quantity=convertListToLower(quantity)
        organization=convertListToLower(organization)
                
        finalVictimList=[]
        finalFlaggedVictim=[]

        finalSuspectList=[]
        finalFlaggedSuspect=[]

        if token_length> 512:
              long_articles=1
              print("Article is longer hence it needs to be broken  down")
              short_article_collection = break_longer_artilces (articleStory.lower(), number_of_short_text )
              for short_article in short_article_collection:
                  #Global variables
                  print("--Processing short article")
                  count=1
                    
                  vic_sus_AnswerList=[]
                  victim_lst,flagged_victim_lst= get_victims_list(short_article, victimSuspectFlag=0)   # 0 for victim
                  finalVictimList.append(victim_lst ) # --
                  finalFlaggedVictim.append(flagged_victim_lst ) #--

                  count=1
                  vic_sus_AnswerList=[]
                  suspect, flagged_suspect=get_victims_list(short_article.lower(), victimSuspectFlag=1) # 1 for TRafficker
                  finalSuspectList.append(suspect)
                  finalFlaggedSuspect.append(flagged_suspect)


              articleStory=short_article_collection[0]
              finalFlaggedVictim= list(chain.from_iterable(finalFlaggedVictim)) 
              finalVictimList=list(chain.from_iterable(finalVictimList)) 

              finalSuspectList= list(chain.from_iterable(finalSuspectList )) 
              finalFlaggedSuspect= list(chain.from_iterable( finalFlaggedSuspect )) 
              
              # Here we have to check if a particular person is in both the Victim and Suspect list or not
              # if so then we need to flag those users and add them to Flagged users list
              if (len(finalVictimList) >1) and (len(finalSuspectList)>1) :
                  print(" BEFORE-finalVictimList->",finalVictimList)
                  print(" compare with suspect list->",finalSuspectList)
                  print("BEFORE finalFlaggedVictim->", finalFlaggedVictim)
                  for v in finalVictimList:
                    for s in finalSuspectList:
                      if v.strip()==s.strip():
                        finalFlaggedVictim.append(v)
                        finalFlaggedSuspect.append(v)
                  
                  finalVictimList= list( set(finalVictimList).difference(set(finalFlaggedVictim)))
                  finalSuspectList= list( set(finalSuspectList).difference(set(finalFlaggedSuspect)))
                  #---------------------------------------------------------------------------------
                  print("AFTER-finalVictimList->",finalVictimList)
                  print("AFTER finalFlaggedVictim->", finalFlaggedVictim)

              # now get the cif fields from all the short articles and select the very best of them
              ##################################-----------------------------
              numberOfVictims=get_numberOfVictims(finalVictimList, short_article_collection[0]) #usually presented in the first part of article

              genderOfVictims={}
              ageOfVictims={}
              nationalityOfVictims={}
              addressOfVictims={}
              occupationOfVictims={}
              educationOfVictims={}
              guardiansOfVictims={}
              for vic in finalVictimList:
                #AGE
                key_lst=[]
                score_lst=[]
                dict_from_fun={}
                print(f" AGE - {vic} ")
                for short in short_article_collection:
                    dict_from_fun=get_ageOfVictims([vic],short)
                    for k,v in dict_from_fun.items():
                      print(k,v)
                      key=v[0]
                      score=v[1]
                    key_lst.append(key)
                    score_lst.append(score)
                print(key_lst,score_lst)
                ageOfVictims[vic]=key_lst[score_lst.index(max(score_lst))]

                
                #GENDER
                key_lst=[]
                score_lst=[]
                dict_from_fun={}
                print(f"GENDER - {vic}")
                for short in short_article_collection:
                    dict_from_fun=get_genderOfVictims([vic],short)
                    for k,v in dict_from_fun.items():
                      print(k,v)
                      key=v[0]
                      score=v[1]
                    key_lst.append(key)
                    score_lst.append(score)
                print(key_lst,score_lst)
                genderOfVictims[vic]=key_lst[score_lst.index(max(score_lst))]

                #NATIONALITY 
                key_lst=[]
                score_lst=[]
                dict_from_fun={}
                print(f"NATIONALITY - {vic}")
                for short in short_article_collection:
                    dict_from_fun=get_nationalityOfVictims([vic],short)
                    for k,v in dict_from_fun.items():
                      print(k,v)
                      key=v[0]
                      score=v[1]
                    key_lst.append(key)
                    score_lst.append(score)
                print(key_lst,score_lst)
                nationalityOfVictims[vic]=key_lst[score_lst.index(max(score_lst))]
                #ADDRESS

                key_lst=[]
                score_lst=[]
                dict_from_fun={}
                print(f" ADDRESS - {vic}")
                for short in short_article_collection:
                    dict_from_fun=get_addressOfVictims([vic],short)
                    for k,v in dict_from_fun.items():
                      print(k,v)
                      key=v[0]
                      score=v[1]
                    key_lst.append(key)
                    score_lst.append(score)
                print(key_lst,score_lst)
                addressOfVictims[vic]=key_lst[score_lst.index(max(score_lst))]
                #OCCUPATION

                key_lst=[]
                score_lst=[]
                dict_from_fun={}
                print(f" OCCUPATION - {vic}")
                for short in short_article_collection:
                    dict_from_fun=get_occupationOfVictims([vic],short)
                    for k,v in dict_from_fun.items():
                      print(k,v)
                      key=v[0]
                      score=v[1]
                    key_lst.append(key)
                    score_lst.append(score)
                print(key_lst,score_lst)
                occupationOfVictims[vic]=key_lst[score_lst.index(max(score_lst))]

                #EDUCATION

                key_lst=[]
                score_lst=[]
                dict_from_fun={}
                print(f" EDUCATION - {vic}")
                for short in short_article_collection:
                    dict_from_fun=get_educationOfVictims([vic],short)
                    for k,v in dict_from_fun.items():
                      print(k,v)
                      key=v[0]
                      score=v[1]
                    key_lst.append(key)
                    score_lst.append(score)
                print(key_lst,score_lst)
                educationOfVictims[vic]=key_lst[score_lst.index(max(score_lst))]

              
              #-- SUSPECT ATTRIBUTES ################################################
              genderOfSuspects={}
              ageOfSuspects={}
              nationalityOfSuspects={}
              addressOfSuspects={}
              occupationOfSuspects={}
              educationOfSuspects={}
              for vic in finalSuspectList: # vic is equivalent to sus here
                #AGE
                key_lst=[]
                score_lst=[]
                dict_from_fun={}
                print(f"AGE - {vic}")
                for short in short_article_collection:
                    dict_from_fun=get_ageOfVictims([vic],short)
                    for k,v in dict_from_fun.items():
                      print(k,v)
                      key=v[0]
                      score=v[1]
                    key_lst.append(key)
                    score_lst.append(score)
                print(key_lst,score_lst)
                ageOfSuspects[vic]=key_lst[score_lst.index(max(score_lst))]
                
                #GENDER
                key_lst=[]
                score_lst=[]
                dict_from_fun={}
                print(f"GENDER - {vic}")
                for short in short_article_collection:
                    dict_from_fun=get_genderOfVictims([vic],short)
                    for k,v in dict_from_fun.items():
                      print(k,v)
                      key=v[0]
                      score=v[1]
                    key_lst.append(key)
                    score_lst.append(score)
                print(key_lst,score_lst)
                genderOfSuspects[vic]=key_lst[score_lst.index(max(score_lst))]

                #NATIONALITY 
                key_lst=[]
                score_lst=[]
                dict_from_fun={}
                print(f"NATIONALITY - {vic}")
                for short in short_article_collection:
                    dict_from_fun=get_nationalityOfVictims([vic],short)
                    for k,v in dict_from_fun.items():
                      print(k,v)
                      key=v[0]
                      score=v[1]
                    key_lst.append(key)
                    score_lst.append(score)
                print(key_lst,score_lst)
                nationalityOfSuspects[vic]=key_lst[score_lst.index(max(score_lst))]
                #ADDRESS

                key_lst=[]
                score_lst=[]
                dict_from_fun={}
                print(f" ADDRESS - {vic}")
                for short in short_article_collection:
                    dict_from_fun=get_addressOfVictims([vic],short)
                    for k,v in dict_from_fun.items():
                      print(k,v)
                      key=v[0]
                      score=v[1]
                    key_lst.append(key)
                    score_lst.append(score)
                print(key_lst,score_lst)
                addressOfSuspects[vic]=key_lst[score_lst.index(max(score_lst))]
                #OCCUPATION

                key_lst=[]
                score_lst=[]
                dict_from_fun={}
                print(f" OCCUPATION - {vic}")
                for short in short_article_collection:
                    dict_from_fun=get_occupationOfVictims([vic],short)
                    for k,v in dict_from_fun.items():
                      print(k,v)
                      key=v[0]
                      score=v[1]
                    key_lst.append(key)
                    score_lst.append(score)
                print(key_lst,score_lst)
                occupationOfSuspects[vic]=key_lst[score_lst.index(max(score_lst))]

                #EDUCATION

                key_lst=[]
                score_lst=[]
                dict_from_fun={}
                print(f" EDUCATION - {vic}")
                for short in short_article_collection:
                    dict_from_fun=get_educationOfVictims([vic],short)
                    for k,v in dict_from_fun.items():
                      print(k,v)
                      key=v[0]
                      score=v[1]
                    key_lst.append(key)
                    score_lst.append(score)
                print(key_lst,score_lst)
                educationOfSuspects[vic]=key_lst[score_lst.index(max(score_lst))]

                


              ##########################---------------------------------    
              
              promiseToVictims=[]
              purposeToTravel=[]
              formOfExploitation=[]
              for short in short_article_collection:
                
                promiseToVictims.append(get_promiseToVictims(finalVictimList,short) )
                purposeToTravel.append(get_purposeToTravel(finalVictimList,short) )
                formOfExploitation.append(get_formOfExploitation(finalVictimList,short) )
              
              


          ##################################----------------------

        else: # if article is shorter in length
                
                count=1
                #victim_list=[]
                vic_sus_AnswerList=[]
                finalVictimList,finalFlaggedVictim=get_victims_list(articleStory, victimSuspectFlag =0) #--
                
                
                count=1
                vic_sus_AnswerList=[]
                finalSuspectList,finalFlaggedSuspect=get_victims_list(articleStory, victimSuspectFlag =1) #--

                # Here we have to check if a particular person is in both the Victim and Suspect list or not
                # if so then we need to flag those users and add them to Flagged users list
                if (len(finalVictimList) >1) and (len(finalSuspectList)>1) :
                    print("BEFORE-finalVictimList->",finalVictimList)
                    print("BEFORE finalFlaggedVictim->", finalFlaggedVictim)
                    print('Comparing with final suspect list ->', finalSuspectList)
                    for v in finalVictimList:
                      for s in finalSuspectList:
                        if v.strip()==s.strip():
                          finalFlaggedVictim.append(v)
                          finalFlaggedSuspect.append(v)
                          print("---------removing-------,",v)
                    
                    finalVictimList= list( set(finalVictimList).difference(set(finalFlaggedVictim)))
                    finalSuspectList= list( set(finalSuspectList).difference(set(finalFlaggedSuspect)))
                    #---------------------------------------------------------------------------------
                    print("AFTER-finalVictimList->",finalVictimList)
                    print("AFTER finalFlaggedVictim->", finalFlaggedVictim)

                #--- Victims Attributes
                long_articles=0
                numberOfVictims=get_numberOfVictims(finalVictimList, articleStory) #digit
                genderOfVictims=get_genderOfVictims(finalVictimList,articleStory) # dict of victims
                ageOfVictims=get_ageOfVictims(finalVictimList,articleStory)
                nationalityOfVictims=get_nationalityOfVictims(finalVictimList,articleStory)
                addressOfVictims=get_addressOfVictims(finalVictimList,articleStory)
                occupationOfVictims=get_occupationOfVictims(finalVictimList,articleStory)
                educationOfVictims=get_educationOfVictims(finalVictimList,articleStory)
                guardiansOfVictims=get_guardiansOfVictims(finalVictimList,articleStory)
                traffickers=get_traffickers(finalVictimList,articleStory)
                promiseToVictims=get_promiseToVictims(finalVictimList,articleStory)
                purposeToTravel=get_purposeToTravel(finalVictimList,articleStory)
                formOfExploitation=get_formOfExploitation(finalVictimList,articleStory)

                
                # ---SUSPECT Parameters
                genderOfSuspects=get_genderOfVictims(finalSuspectList,articleStory) # dict of Suspects
                ageOfSuspects=get_ageOfVictims(finalSuspectList,articleStory)
                nationalityOfSuspects=get_nationalityOfVictims(finalSuspectList,articleStory)
                addressOfSuspects=get_addressOfVictims(finalSuspectList,articleStory)
                occupationOfSuspects=get_occupationOfVictims(finalSuspectList,articleStory)
                educationOfSuspects=get_educationOfVictims(finalSuspectList,articleStory)

                



        


       
        # 1.x get the other  answers

        #processing results to save in the df
        #result_dict={}
        result_dict["len_token"]=token_length
        result_dict["article"]=articleStory
        result_dict["victim_list"]=finalVictimList
        result_dict["flagged_victim_list"]=finalFlaggedVictim
        result_dict["len_victim_list"]=len(finalVictimList)
        result_dict["num_of_victims"]=numberOfVictims
        result_dict["long_articles"]=long_articles


        for id in range(len(finalVictimList)) :
            result_dict[f"pv{id+1}_name"]=finalVictimList[id]

        for id, k in zip(range(len(genderOfVictims)), genderOfVictims.items()):            
            try:
                result_dict[f"pv{id+1}_gender"]=k[1][0]
            except:
                result_dict[f"pv{id+1}_gender"]=k[1]

        for id, k in zip(range(len(ageOfVictims)), ageOfVictims.items()):           
            try: 
                result_dict[f"pv{id+1}_age"]=k[1][0]
            except:
                result_dict[f"pv{id+1}_age"]=k[1]

        for id, k in zip(range(len(nationalityOfVictims)), nationalityOfVictims.items()):            
            try:
                result_dict[f"pv{id+1}_nationality"]=k[1][0]
            except:
                result_dict[f"pv{id+1}_nationality"]=k[1]

        for id, k in zip(range(len(occupationOfVictims)), occupationOfVictims.items()):            
            try:
                result_dict[f"pv{id+1}_education"]=k[1][0]
            except:
                result_dict[f"pv{id+1}_education"]=k[1]

        for id, k in zip(range(len(addressOfVictims)), addressOfVictims.items()):           
            try:
                result_dict[f"pv{id+1}_address"]=k[1][0]
            except:
                result_dict[f"pv{id+1}_address"]=k[1]

        for id, k in zip(range(len(educationOfVictims)), educationOfVictims.items()):            
            try:
                result_dict[f"pv{id+1}_occupation"]=k[1][0]
            except:
                result_dict[f"pv{id+1}_occupation"]=k[1]

        for id, k in zip(range(len(guardiansOfVictims)), guardiansOfVictims.items()):            
            try:
                result_dict[f"pv{id+1}_guardian"]=k[1][0]
            except:
                result_dict[f"pv{id+1}_guardian"]=k[1]

        
        #--suspect parameter
      
        result_dict['suspect_list']=finalSuspectList
        result_dict['flagged_suspect_list']=finalFlaggedSuspect

        for id in range(len(finalSuspectList)) :
            result_dict[f"pb{id+1}_name"]=finalSuspectList[id]

        for id, k in zip(range(len(genderOfSuspects)), genderOfSuspects.items()):            
            try:
                result_dict[f"pb{id+1}_gender"]=k[1][0]
            except:
                result_dict[f"pb{id+1}_gender"]=k[1]
                

        for id, k in zip(range(len(ageOfSuspects)), ageOfSuspects.items()):           
            try:
                result_dict[f"pb{id+1}_age"]=k[1][0]
            except:
                result_dict[f"pb{id+1}_age"]=k[1]

        for id, k in zip(range(len(nationalityOfSuspects)), nationalityOfSuspects.items()):            
            try:
                result_dict[f"pb{id+1}_nationality"]=k[1][0]
            except:
                result_dict[f"pb{id+1}_nationality"]=k[1]

        for id, k in zip(range(len(occupationOfVictims)), occupationOfVictims.items()):            
            try: 
                result_dict[f"pb{id+1}_education"]=k[1][0]
            except:
                result_dict[f"pb{id+1}_education"]=k[1]

        for id, k in zip(range(len(addressOfSuspects)), addressOfSuspects.items()):           
            try:
                result_dict[f"pb{id+1}_address"]=k[1][0]
            except:
                result_dict[f"pb{id+1}_address"]=k[1]

        for id, k in zip(range(len(educationOfSuspects)), educationOfSuspects.items()):            
            try:
                result_dict[f"pb{id+1}_occupation"]=k[1][0]
            except:
                result_dict[f"pb{id+1}_occupation"]=k[1]



        ###############################################
        
        result_dict["promise_to_victims"]= promiseToVictims  
        result_dict["purpose_to_travel"]= purposeToTravel
        result_dict["form_of_exploitation"]= formOfExploitation

        # Article Metadata
        
        end_time = datetime.datetime.now()
        result_dict["time_taken"]=((end_time-start_time).total_seconds())/60
        result_dict["cif_extracted_flag"]=1
        
        print(f"The total time it took in minute to run it, is: :",((end_time-start_time).total_seconds())/60)

        
        idx=id
        print("Victim List: ", result_dict["victim_list"],"\n",
                "Flagged Victim List: ",result_dict["flagged_victim_list"] ,"\n")
        
        
        result_df=result_df.append(result_dict,ignore_index= True)
        
        
        result_df.to_csv("result_df.csv")
               
        
        print(f"------------****-------------- ENDS--------****-----------------------***--------------------------\n")
        